In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

! pip install lxml

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

In [122]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Get wiki page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M data

In [123]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [124]:
wiki_data = soup.find('table',{'class':'wikitable sortable'})
#wiki_data

Crate empty dataframe to fill with wiki_data

In [125]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df1 = pd.DataFrame(columns=column_names)
print("shape",df1.shape)
df1

shape (0, 3)


,PostalCode,Borough,Neighborhood


Read table from url and assume it is the first and only table in the url

In [126]:
wiki_data = soup.find('table',{'class':'wikitable sortable'})
table = wiki_data
rows = table.find_all("tr")
for row in rows:
        columns = row.find_all("td")
        headers = row.find_all("th")
        if len(columns) == 0 : continue
        Postcode = columns[0].text
        Borough = columns[1].text
        Neighbourhood = columns[2].text#.split("/a")
        df1 = df1.append({'PostalCode': Postcode,'Borough': Borough,'Neighborhood': Neighbourhood}, ignore_index=True)
#clean table       
df1 = df1.replace(r'\n','', regex=True) 
df1.drop(df1.index[0],inplace=True)

print("shape",df1.shape,"type",type(df1))
df1.tail()

shape (288, 3) type <class 'pandas.core.frame.DataFrame'>


,PostalCode,Borough,Neighborhood
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [127]:
df1=df1[df1.Borough != 'Not assigned']
print("shape",df1.shape)
df1.tail()

shape (212, 3)


,PostalCode,Borough,Neighborhood
283,M8Z,Etobicoke,Kingsway Park South West
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor


More than one neighborhood can exist in one postal code area. 
Group rows with duplicate PostalCode The Neighborhoods will be separated with a comma.

In [131]:
df1 = df1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('shape',df1.shape)

shape (103, 3)


,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


Confirm Postalcode M5A

In [ ]:
df1[df1['PostalCode']== 'M5A']

Not assigned neighborhood cell will be the same as the borough cell

In [133]:
df1['Neighborhood'] = df1.apply(
    lambda row: row['Borough'] if (row['Neighborhood']== 'Not assigned') else row['Neighborhood'],
    axis=1
)
df1[df1['PostalCode']== 'M7A']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [130]:
df1.shape

(103, 3)